In [1]:
import pandas as pd
df = pd.read_csv('data/AllVideoInfo.csv')
df['pubdate'] = pd.to_datetime(df['pubdate'])

df.shape

(657, 12)

### 图1 播放量前10的视频

获取播放量前10的视频

In [3]:
top_10_views = df.nlargest(10, 'view')
top_10_views

,bvid,uid,title,tname,pubdate,view,danmaku,reply,favorite,coin,share,like
160,BV11i4y1L7QQ,23947287,苏联为何而强大？【小约翰】,人文历史,2020-11-14 17:12:00,27533436,88189,33057,461126,825530,210328,1062598
142,BV1qi4y1A76i,23947287,人 间 之 屑 卡 扎 菲【奇葩小国18】,人文历史,2021-04-24 10:49:36,19271838,52630,9287,83507,169516,60128,433090
143,BV1ro4y1f7x4,23947287,偷袭救不了阿根廷老铁【奇葩小国17】,人文历史,2021-04-17 10:52:06,14860611,52025,8958,57040,207760,27639,358968
104,BV16F411B744,23947287,最能让带英破防的人是谁？【硬核狠人21】,人文历史,2021-12-25 11:00:06,14532086,67419,18042,153650,467306,73275,607016
131,BV11L411H7o7,23947287,非洲的切格瓦拉是谁？【奇葩小国25】,人文历史,2021-07-24 11:00:00,14339153,88426,28854,172110,488642,113069,546099
313,BV1Z5411p7Ct,519872016,壮阔史诗！二战历史年表，超燃影视化剪辑【历史调研室】,人文历史,2020-06-09 17:30:40,14243043,70308,14981,553572,559036,151321,632616
120,BV1Pq4y1o7sV,23947287,耶路撒冷究竟是谁老家？【奇葩小国28】,人文历史,2021-09-26 13:23:10,14053514,89368,28561,90340,254292,57246,423942
159,BV1Uy4y1S7Eq,23947287,如何用外交得罪所有大国？【奇葩小国01】,人文历史,2020-12-11 14:21:30,13463863,40280,4161,44665,100487,29254,350905
140,BV1kA411G7jg,23947287,古 巴 人 民 真 蒸 汽【奇葩小国19】,人文历史,2021-05-13 12:16:29,13326279,56409,11431,97660,285017,43578,414977
246,BV1ke4y1h7VJ,504934876,深度|| 为了让悟空脱离低级趣味，佛祖究竟花了多少经费？（中秋特供）,人文历史,2022-09-09 09:00:00,12957058,35239,16502,380709,979630,170147,909200


In [4]:
from pyecharts.charts import Bar, Line
from pyecharts import options as opts
from pyecharts.globals import ThemeType
import textwrap

# 截断标题函数
def truncate_title(title, length=15):
    return title if len(title) <= length else title[:length] + '...'

# 截断标题并换行
top_10_views['short_title'] = top_10_views['title'].apply(lambda x: '\n'.join(textwrap.wrap(truncate_title(x), width=8)))

# 创建美化的播放量前10的柱状图和点赞数的折线图
bar = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
    .add_xaxis(top_10_views['short_title'].tolist())
    .add_yaxis("播放量", top_10_views['view'].tolist())
    .extend_axis(
        yaxis=opts.AxisOpts(
            name="点赞数/分享数等",
            type_="value",
            axislabel_opts=opts.LabelOpts(formatter="{value}"),
        )
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="播放量前10的视频数据",
            title_textstyle_opts=opts.TextStyleOpts(font_size=20),
            pos_left="center"  # 让标题居中
        ),
        xaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(rotate=-45)  # 不进行标签旋转
        ),
        yaxis_opts=opts.AxisOpts(
            name="播放量",
            axislabel_opts=opts.LabelOpts(formatter="{value}"),
            splitline_opts=opts.SplitLineOpts(is_show=True)
        ),
        legend_opts=opts.LegendOpts(
            pos_right="20%",  # 调整图例距离右边的距离
            pos_top="5%"  # 调整图例距离顶部的距离
        ),  # 设置图注放右边
        toolbox_opts=None,  # 移除所有控制按钮
        datazoom_opts=None,
        tooltip_opts=opts.TooltipOpts(
            is_show=True, 
            trigger="axis", 
            axis_pointer_type="shadow",
            formatter="{a}: {b} <br/>{c}次"
        )
    )
    .set_series_opts(
        label_opts=opts.LabelOpts(
            is_show=True, 
            position="inside",  # 将标签位置调整为柱状图内部
            formatter="{c}"
        ),
    )
)

line_like = (
    Line()
    .add_xaxis(top_10_views['short_title'].tolist())
    .add_yaxis(
        series_name="点赞数",
        yaxis_index=1,
        y_axis=top_10_views['like'].tolist(),
        label_opts=opts.LabelOpts(is_show=False),
        z=2,  # 设置折线图的 Z 轴层叠顺序为 2，确保在柱状图上方显示
        symbol='circle',  # 设置数据点形状为圆圈
        symbol_size=10,  # 设置数据点大小为 8
        linestyle_opts=opts.LineStyleOpts(color='red')  # 设置折线颜色和柱状图标记点的颜色一致
    )
)

line_favorite = (
    Line()
    .add_xaxis(top_10_views['short_title'].tolist())
    .add_yaxis(
        series_name="收藏量",
        yaxis_index=1,
        y_axis=top_10_views['favorite'].tolist(),
        label_opts=opts.LabelOpts(is_show=False),
        z=2,  # 设置折线图的 Z 轴层叠顺序为 2，确保在柱状图上方显示
        symbol='circle',  # 设置数据点形状为圆圈
        symbol_size=10,  # 设置数据点大小为 8
        linestyle_opts=opts.LineStyleOpts(color='rgba(255, 69, 0, 0.7)')  # 设置折线颜色为橙色
    )
)

line_share = (
    Line()
    .add_xaxis(top_10_views['short_title'].tolist())
    .add_yaxis(
        series_name="分享数",
        yaxis_index=1,
        y_axis=top_10_views['share'].tolist(),
        label_opts=opts.LabelOpts(is_show=False),
        z=2, # 设置折线图的 Z 轴层叠顺序为 2，确保在柱状图上方显示
        symbol='circle', # 设置数据点形状为圆圈
        symbol_size=10, # 设置数据点大小为 8
        linestyle_opts=opts.LineStyleOpts(color='rgba(255, 15, 0, 0.7)') 
    )
)

line_danmaku = (
    Line()
    .add_xaxis(top_10_views['short_title'].tolist())
    .add_yaxis(
        series_name="弹幕数",
        yaxis_index=1,
        y_axis=top_10_views['danmaku'].tolist(),
        label_opts=opts.LabelOpts(is_show=False),
        z=2, # 设置折线图的 Z 轴层叠顺序为 2，确保在柱状图上方显示
        symbol='circle', # 设置数据点形状为圆圈
        symbol_size=10, # 设置数据点大小为 8
        linestyle_opts=opts.LineStyleOpts(color='rgba(255, 215, 0, 0.7)') # 设置折线颜色为金黄色
    )
)
line_reply = (
    Line()
    .add_xaxis(top_10_views['short_title'].tolist())
    .add_yaxis(
        series_name="评论数",
        yaxis_index=1,
        y_axis=top_10_views['reply'].tolist(),
        label_opts=opts.LabelOpts(is_show=False),
        z=2, # 设置折线图的 Z 轴层叠顺序为 2，确保在柱状图上方显示
        symbol='circle', # 设置数据点形状为圆圈
        symbol_size=10, # 设置数据点大小为 8
        linestyle_opts=opts.LineStyleOpts(color='rgba(255, 215, 0, 0.7)') # 设置折线颜色为金黄色
    )
)
line_coin = (
    Line()
    .add_xaxis(top_10_views['short_title'].tolist())
    .add_yaxis(
        series_name="投币数",
        yaxis_index=1,
        y_axis=top_10_views['coin'].tolist(),
        label_opts=opts.LabelOpts(is_show=False),
        z=2, # 设置折线图的 Z 轴层叠顺序为 2，确保在柱状图上方显示
        symbol='circle', # 设置数据点形状为圆圈
        symbol_size=10, # 设置数据点大小为 8
        linestyle_opts=opts.LineStyleOpts(color='rgba(255, 215, 0, 0.7)') # 设置折线颜色为金黄色
    )
)
# 先添加柱状图，再添加折线图
bar.overlap(line_like)
bar.overlap(line_favorite)
bar.overlap(line_share)
bar.overlap(line_danmaku)
bar.overlap(line_reply)
bar.overlap(line_coin)


# 保存为HTML文件
bar.render("page/top10.html")

'c:\\Users\\ZZH\\Desktop\\1\\page\\top10.html'

### 图2 视频分类分布

处理分类数据，将数量小于20的分类合并为“其他”

In [66]:
import pandas as pd
df = pd.read_csv('data/data01.csv')
# 统计每个分类的视频数量
category_counts = df['tname'].value_counts()

# 将数量小于20的分类合并为“其他”
df['tname'] = df['tname'].apply(lambda x: x if category_counts[x] >= 20 else '其他')

# 重新统计分类数量
category_counts = df['tname'].value_counts()
category_counts

tname
社会      1473
热点       860
综合       505
环球       351
其他       216
日常       146
科学科普      72
人文历史      67
音乐现场      36
足球        36
竞技体育      26
科工机械      21
Name: count, dtype: int64

In [70]:
from pyecharts.charts import Pie
import pyecharts.options as opts

# 将 category_counts 转换为列表
t_num = category_counts.values.tolist()
t_lab = category_counts.index.tolist()

# 创建饼图对象
pie = (
    Pie(init_opts=opts.InitOpts(width='720px', height='320px'))
    .add(series_name='', data_pair=[(i, j) for i, j in zip(t_lab, t_num)])
    .set_global_opts(
        title_opts=opts.TitleOpts(title="视频分类分布",
                                  title_textstyle_opts=opts.TextStyleOpts(font_size=14), 
                                  pos_left='center'),
        legend_opts=opts.LegendOpts(orient='vertical', pos_left='right'),
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
)

# 渲染图表
pie.render("page/tname.html")


'c:\\Users\\ZZH\\Desktop\\数据可视化\\page\\tname.html'

每类平均播放量

In [72]:
import pandas as pd
df = pd.read_csv('data/data01.csv')

In [75]:
# 按视频类别分组，计算每类视频的总观看量和视频数量
grouped_df = df.groupby('tname').agg({'view': 'sum', 'bvid': 'count'}).reset_index()
grouped_df.rename(columns={'bvid': 'video_count'}, inplace=True)

# 计算每类视频的平均观看量并取整
grouped_df['average_view'] = (grouped_df['view'] / grouped_df['video_count'] / 10000).round(2)  # 播放量除以 10000，并保留两位小数
grouped_df['view'] = (grouped_df['view'] / 10000).round(2) 

# 排序并取出平均播放量前10的视频
top_10_videos = grouped_df.nlargest(10, 'average_view')

In [77]:
# 创建条形图
bar = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
    .add_xaxis(top_10_videos['tname'].tolist())
    .add_yaxis("平均播放量（万次）", top_10_videos['average_view'].tolist(), label_opts=opts.LabelOpts(formatter="{c}"))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="平均播放量前10的视频", pos_left='center'),
        xaxis_opts=opts.AxisOpts(type_="category"),
        yaxis_opts=opts.AxisOpts(
            name="平均播放量（万次）",
            axislabel_opts=opts.LabelOpts(formatter="{value}"),
        ),
        legend_opts=opts.LegendOpts(
            pos_right="10%",  # 调整图例距离右边的距离
            pos_top="5%"  # 调整图例距离顶部的距离
        ),  # 设置图注放右边
    )
)

# 渲染图表，可以生成 HTML 文件查看
bar.render("page/tname_top10.html")

'c:\\Users\\ZZH\\Desktop\\数据可视化\\page\\tname_top10.html'

In [68]:
# 统计每个分类的视频数量
category_counts = df['tname'].value_counts()

# 将数量小于20的分类合并为“其他”
df['tname'] = df['tname'].apply(lambda x: x if category_counts[x] >= 20 else '其他')

# 重新统计分类数量
category_counts = df['tname'].value_counts()

# 按视频类别分组，计算每类视频的总观看量和视频数量
grouped_df = df.groupby('tname').agg({'view': 'sum', 'bvid': 'count'}).reset_index()
grouped_df.rename(columns={'bvid': 'video_count'}, inplace=True)

# 计算每类视频的平均观看量并取整
grouped_df['average_view'] = (grouped_df['view'] / grouped_df['video_count'] / 10000).round(2)  # 播放量除以 10000，并保留两位小数
grouped_df['view'] = (grouped_df['view'] / 10000).round(2) 

In [70]:
bar = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
    .add_xaxis(grouped_df['tname'].tolist())
    .add_yaxis("播放量", grouped_df['view'].tolist())
    .extend_axis(
        yaxis=opts.AxisOpts(
            name="平均播放量（万次）",
            type_="value",
            axislabel_opts=opts.LabelOpts(formatter="{value}"),
        )
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="视频类别对应的观看量",
            title_textstyle_opts=opts.TextStyleOpts(font_size=20),
            pos_left="center"  # 让标题居中
        ),
        xaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(rotate=-25)  # 进行标签旋转
        ),
        yaxis_opts=opts.AxisOpts(
            name="播放量（万次）",
            axislabel_opts=opts.LabelOpts(formatter="{value}"),
            splitline_opts=opts.SplitLineOpts(is_show=True)
        ),
        legend_opts=opts.LegendOpts(
            pos_right="20%",  # 调整图例距离右边的距离
            pos_top="5%"  # 调整图例距离顶部的距离
        ),  # 设置图注放右边
        toolbox_opts=None,  # 移除所有控制按钮
        tooltip_opts=opts.TooltipOpts(
            is_show=True, 
            trigger="axis", 
            axis_pointer_type="shadow",
            formatter="{a}: {b} <br/>{c}万次"
        ),
        datazoom_opts=[opts.DataZoomOpts(type_="slider")],  # 添加滑块
    )
    .set_series_opts(
        label_opts=opts.LabelOpts(
            is_show=True, 
            position="inside",  # 将标签位置调整为柱状图内部
            formatter="{c}"
        ),
    )
)
line_average = (
    Line()
    .add_xaxis(grouped_df['tname'].tolist())
    .add_yaxis(
        series_name="平均播放量（万次）",
        yaxis_index=1,
        y_axis=grouped_df['average_view'].tolist(),
        label_opts=opts.LabelOpts(is_show=False),
        z=2,  # 设置折线图的 Z 轴层叠顺序为 2，确保在柱状图上方显示
        symbol='circle',  # 设置数据点形状为圆圈
        symbol_size=10,  # 设置数据点大小为 8
        linestyle_opts=opts.LineStyleOpts(color='red')  # 设置折线颜色和柱状图标记点的颜色一致
    )
)
bar.overlap(line_average)
bar.render("page\\tname_views.html")

'c:\\Users\\ZZH\\Desktop\\数据可视化\\page\\tname_views.html'

### 图3 日期——播放量

In [5]:
from pyecharts import options as opts
from pyecharts.charts import Line

# 提取需要的列并进行数据处理
df['pubdate'] = pd.to_datetime(df['pubdate'])  # 将日期列转换为 datetime 类型
df['date'] = df['pubdate'].dt.strftime('%Y-%m-%d')  # 提取日期部分

# 按日期分组并计算观看量总和
grouped_df = df.groupby('date').agg({'view': 'sum'}).reset_index()

# 提取 X 轴和 Y 轴数据
dates = grouped_df['date'].tolist()
views = grouped_df['view'].tolist()

# 创建 Line 图表
line_chart = (
    Line()
    .add_xaxis(dates)  # X 轴为日期
    .add_yaxis("观看量", views, markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]))  # Y 轴为观看量，标记最大值
    .set_global_opts(title_opts=opts.TitleOpts(title="视频发布日期对观看量的影响", pos_left='center'),  # 设置标题居中
                     xaxis_opts=opts.AxisOpts(name="日期", type_="category", boundary_gap=False),  # 设置 X 轴名称
                     yaxis_opts=opts.AxisOpts(name="观看量"),  # 设置 Y 轴名称
                     legend_opts=opts.LegendOpts(
                        pos_right="20%",  # 调整图例距离右边的距离
                        pos_top="5%"  # 调整图例距离顶部的距离
                    ),  # 设置图注放右边
                    #  tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross")  # 设置提示框
     )                
)

# 渲染图表，可以生成 HTML 文件查看
line_chart.render("page/date_views1.html")

'c:\\Users\\ZZH\\Desktop\\1\\page\\date_views1.html'

In [6]:
import pandas as pd

# 读取 CSV 文件
df = pd.read_csv('data/AllVideoInfo.csv')

数据处理

In [7]:
# 提取需要的列并进行数据处理
df['pubdate'] = pd.to_datetime(df['pubdate'])  # 将日期列转换为 datetime 类型
df['month'] = df['pubdate'].dt.strftime('%Y-%m')  # 提取月份部分

# 按月份分组，计算每月的总观看量和视频数量
grouped_df = df.groupby('month').agg({'view': 'sum', 'bvid': 'count'}).reset_index()
grouped_df.rename(columns={'bvid': 'video_count'}, inplace=True)

# 计算每月的平均观看量并取整
grouped_df['average_view'] = (grouped_df['view'] / grouped_df['video_count'] / 10000).round(2)  # 播放量除以 10000，并保留两位小数
grouped_df['view'] = (grouped_df['view']/ 10000).round(2) 

In [8]:
from pyecharts.charts import Bar, Line
from pyecharts import options as opts
from pyecharts.globals import ThemeType

# 假设您已经定义了 grouped_df 和 grouped_df['average_view']

bar = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
    .add_xaxis(grouped_df['month'].tolist())
    .add_yaxis("播放量", grouped_df['view'].tolist())
    .extend_axis(
        yaxis=opts.AxisOpts(
            name="平均播放量（万次）",
            type_="value",
            axislabel_opts=opts.LabelOpts(formatter="{value}"),
        )
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="视频发布日期对观看量的影响",
            title_textstyle_opts=opts.TextStyleOpts(font_size=20),
            pos_left="center"  # 让标题居中
        ),
        xaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(rotate=0)  # 进行标签旋转
        ),
        yaxis_opts=opts.AxisOpts(
            name="播放量（万次）",
            axislabel_opts=opts.LabelOpts(formatter="{value}"),
            splitline_opts=opts.SplitLineOpts(is_show=True)
        ),
        legend_opts=opts.LegendOpts(
            pos_right="20%",  # 调整图例距离右边的距离
            pos_top="5%"  # 调整图例距离顶部的距离
        ),  # 设置图注放右边
        toolbox_opts=None,  # 移除所有控制按钮
        tooltip_opts=opts.TooltipOpts(
            is_show=True, 
            trigger="axis", 
            axis_pointer_type="shadow",
            formatter="{a}: {b} <br/>{c}万次"
        ),
        datazoom_opts=[opts.DataZoomOpts(type_="slider")],  # 添加滑块
    )
    .set_series_opts(
        label_opts=opts.LabelOpts(
            is_show=True, 
            position="inside",  # 将标签位置调整为柱状图内部
            formatter="{c}"
        ),
    )
)
line_average = (
    Line()
    .add_xaxis(grouped_df['month'].tolist())
    .add_yaxis(
        series_name="平均播放量",
        yaxis_index=1,
        y_axis=grouped_df['average_view'].tolist(),
        label_opts=opts.LabelOpts(is_show=False),
        z=2,  # 设置折线图的 Z 轴层叠顺序为 2，确保在柱状图上方显示
        symbol='circle',  # 设置数据点形状为圆圈
        symbol_size=10,  # 设置数据点大小为 8
        linestyle_opts=opts.LineStyleOpts(color='red')  # 设置折线颜色和柱状图标记点的颜色一致
    )
)
bar.overlap(line_average)
bar.render("page\date_views.html")


'c:\\Users\\ZZH\\Desktop\\1\\page\\date_views.html'

### 图4 时刻——播放量

In [9]:
import pandas as pd

# 读取 CSV 文件
df = pd.read_csv('data/AllVideoInfo.csv')

In [10]:
# 将日期列转换为时间对象
df['pubdate'] = pd.to_datetime(df['pubdate'])

# 将一天划分为24个区间
df['hour'] = df['pubdate'].dt.hour.astype(int)  # 将小时部分转换为整数

# 按小时分组，计算每小时的总观看量和视频数量
hourly_grouped_df = df.groupby('hour').agg({'view': 'sum', 'bvid': 'count'}).reset_index()
hourly_grouped_df.rename(columns={'bvid': 'video_count'}, inplace=True)

# 计算每小时的平均观看量并取整
hourly_grouped_df['average_view'] = (hourly_grouped_df['view'] / hourly_grouped_df['video_count'] / 10000).round(2)  # 播放量除以 10000，并保留两位小数
hourly_grouped_df['view'] = (hourly_grouped_df['view'] / 10000).round(2) 


In [11]:
from pyecharts.charts import Bar, Line
from pyecharts import options as opts
from pyecharts.globals import ThemeType


bar = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
    .add_xaxis(hourly_grouped_df['hour'].tolist())
    .add_yaxis("播放量", hourly_grouped_df['view'].tolist())
    .extend_axis(
        yaxis=opts.AxisOpts(
            name="平均播放量（万次）",
            type_="value",
            axislabel_opts=opts.LabelOpts(formatter="{value}"),
        )
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="视频发布时间对观看量的影响",
            title_textstyle_opts=opts.TextStyleOpts(font_size=20),
            pos_left="center"  # 让标题居中
        ),
        xaxis_opts=opts.AxisOpts(
            axislabel_opts=opts.LabelOpts(rotate=0)  # 进行标签旋转
        ),
        yaxis_opts=opts.AxisOpts(
            name="播放量（万次）",
            axislabel_opts=opts.LabelOpts(formatter="{value}"),
            splitline_opts=opts.SplitLineOpts(is_show=True)
        ),
        legend_opts=opts.LegendOpts(
            pos_right="20%",  # 调整图例距离右边的距离
            pos_top="5%"  # 调整图例距离顶部的距离
        ),  # 设置图注放右边
        toolbox_opts=None,  # 移除所有控制按钮
        tooltip_opts=opts.TooltipOpts(
            is_show=True, 
            trigger="axis", 
            axis_pointer_type="shadow",
            formatter="{a}: {b} <br/>{c}万次"
        ),
        datazoom_opts=[opts.DataZoomOpts(type_="slider")],  # 添加滑块
    )
    .set_series_opts(
        label_opts=opts.LabelOpts(
            is_show=True, 
            position="inside",  # 将标签位置调整为柱状图内部
            formatter="{c}"
        ),
    )
)
line_average = (
    Line()
    .add_xaxis(hourly_grouped_df['hour'].tolist())
    .add_yaxis(
        series_name="平均播放量（万次）",
        yaxis_index=1,
        y_axis=hourly_grouped_df['average_view'].tolist(),
        label_opts=opts.LabelOpts(is_show=False),
        z=2,  # 设置折线图的 Z 轴层叠顺序为 2，确保在柱状图上方显示
        symbol='circle',  # 设置数据点形状为圆圈
        symbol_size=10,  # 设置数据点大小为 8
        linestyle_opts=opts.LineStyleOpts(color='red')  # 设置折线颜色和柱状图标记点的颜色一致
    )
)
bar.overlap(line_average)
bar.render("page\\time_views.html")


'c:\\Users\\ZZH\\Desktop\\1\\page\\time_views.html'